In [1]:
import os
import pandas as pd
import re
import numpy as np

In [2]:
df = pd.read_csv("../dataset/nlbse-2023-issue-report.csv")
df.shape
df = df[df['labels']=='bug']
print(df.shape)
df_unique = df.drop_duplicates(subset='id', keep='first')
print(df_unique.shape)
df = df_unique
df["id"] = df["id"].astype(int)
df_selected = df[['id','body']]

concatenated_df = pd.read_csv("../dataset/Manual-labelled-data-25K.csv")
concatenated_df_selected = concatenated_df[['Issue_id','Candidate_String', 'is_secret_human_label']]

# Rename columns for consistency
df_selected = df_selected.rename(columns={'body': 'text'})
concatenated_df_selected = concatenated_df_selected.rename(columns={'Candidate_String': 'candidate_string', 'is_secret_human_label': 'label'})

# Merge the two DataFrames on a common key, for example, 'id'
# Adjust the merge key as needed based on your data
merged_df = concatenated_df_selected.merge(df_selected, left_on='Issue_id', right_on='id')
print(merged_df.shape)
columns_to_remove = ['id']
merged_df.drop(columns=columns_to_remove, inplace=True)
print(merged_df.columns)



C:\Users\ahmed\AppData\Local\Temp\ipykernel_21008\1832453507.py:1: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../dataset/nlbse-2023-issue-report.csv")


(745732, 6)
(726503, 6)
(25451, 5)
Index(['Issue_id', 'candidate_string', 'label', 'text'], dtype='object')


In [3]:
def preprocess(text):
        input_string =  text   
        input_string = re.sub(r'[\'"\│]', '', input_string)
        dir_list_clean = re.sub(r'drwx[-\s]*\d+\s+\w+\s+\w+\s+\d+\s+\w+\s+\d+\s+[0-9a-fA-F-]+.*','',input_string)
        shell_code_free_text = re.sub(r'```shell([^`]+)```','',dir_list_clean,flags=re.IGNORECASE)
        shell_code_free_text = re.sub(r'```Shell\s*"([^"]*)"\s*```','',shell_code_free_text,flags=re.IGNORECASE)
        # saved_game_free_text = re.sub(r'```([^`]+)```','',shell_code_free_text) #etay jhamela hobe
        saved_game_free_text = re.sub(r'<details><summary>Saved game</summary>\n\n```(.*?)```', '', shell_code_free_text)
        remove_packages = re.sub(r'(\w+\.)+\w+','',saved_game_free_text)
        java_exp_free_text = re.sub(r'at\s[\w.$]+\.([\w]+)\(([^:]+:\d+)\)','',remove_packages)
        # url_free_text= re.sub(https?://[^\s#]+#[A-Za-z0-9\-]+,'', java_exp_free_text, flags=re.IGNORECASE)
        url_with_fragment_text= re.sub(r'https?://[^\s#]+#[A-Za-z0-9\-\=\+]+','', java_exp_free_text, flags=re.IGNORECASE)
        url_free_text= re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '',url_with_fragment_text)
        commit_free_text= re.sub(r'commit[ ]?(?:id)?[ ]?[:]?[ ]?([0-9a-f]{40})\b', '', url_free_text, flags=re.IGNORECASE)
        file_path_free_text = re.sub(r"/[\w/. :-]+",'',commit_free_text)
        file_path_free_text = re.sub( r'(/[^/\s]+)+','',file_path_free_text)
        sha256_free_text = re.sub(r'sha256\s*[:]?[=]?\s*[a-fA-F0-9]{64}','',file_path_free_text)
        sha1_free_text = re.sub(r'git-tree-sha1\s*=\s*[a-fA-F0-9]+','',sha256_free_text)
        build_id_free_text = re.sub(r'build-id\s*[:]?[=]?\s*([a-fA-F0-9]+)','',sha1_free_text)
        guids_free_text = re.sub(r'GUIDs:\s+([0-9a-fA-F-]+\s+[0-9a-fA-F-]+\s+[0-9a-fA-F-]+)','',build_id_free_text)
        uuids_free_text = re.sub(r'([0-9a-fA-F-]+\s*,\s*[0-9a-fA-F-]+\s*,\s*[0-9a-fA-F-]+)','',guids_free_text)
        event_id_free_text = re.sub(r'<([^>]+)>','',uuids_free_text)
        UUID_free_text = re.sub(r'(?:UUID|GUID|version|id)[\\=:"\'\s]*\b[a-fA-F0-9]{8}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{12}\b'
,'',event_id_free_text,flags=re.IGNORECASE) ##without the prefix so many false positives can be omitted
        hex_free_text = re.sub(r'(?:data|address|id)[\\=:"\'\s]*\b0x[0-9a-fA-F]+\b','',UUID_free_text,flags=re.IGNORECASE) ## deleting hex ids directly can cause issues
        ss_free_text = re.sub(r'Screenshot_(\d{4}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\w+)','',hex_free_text,flags=re.IGNORECASE)
        cleaned_text = ss_free_text
        return cleaned_text
    

In [4]:
count=0
inverse_count=0
#print(preprocessed_df['modified_text'][2])
for i in range(merged_df.shape[0]):
  #print(i)
  main_string=merged_df['text'][i]
  substring=merged_df['candidate_string'][i]
  if substring in main_string:
    count+=1
  else:
    inverse_count+=1
    merged_df['text'][i] = preprocess(merged_df["text"][i])
print("Mismatch: "+str(inverse_count))
print("Ok: "+str(count))

C:\Users\ahmed\AppData\Local\Temp\ipykernel_21008\3443848363.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['text'][i] = preprocess(merged_df["text"][i])
C:\Users\ahmed\AppData\Local\Temp\ipykernel_21008\3443848363.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['text'][i] = preprocess(merged_df["text"][i])
C:\Users\ahmed\AppData\Local\Temp\ipykernel_21008\3443848363.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

Mismatch: 1807
Ok: 23644


C:\Users\ahmed\AppData\Local\Temp\ipykernel_21008\3443848363.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['text'][i] = preprocess(merged_df["text"][i])
C:\Users\ahmed\AppData\Local\Temp\ipykernel_21008\3443848363.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['text'][i] = preprocess(merged_df["text"][i])
C:\Users\ahmed\AppData\Local\Temp\ipykernel_21008\3443848363.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [5]:
#print(merged_df.size)
#print(merged_df.shape[0])
#print(type(merged_df))
#print(merged_df.head())
preprocessed_df=None
count=0
inverse_count=0
for i in range(merged_df.shape[0]):
  main_string=merged_df['text'][i]
  substring=merged_df['candidate_string'][i]
  if substring not in main_string:
    count+=1
  else:
    inverse_count+=1
    # Append the row to preprocessed_df
    if preprocessed_df is None:
      preprocessed_df = pd.DataFrame(merged_df.iloc[i, :]).transpose()
    else:
      preprocessed_df = pd.concat([preprocessed_df, pd.DataFrame(merged_df.iloc[i, :]).transpose()], ignore_index=True)

print("Mismatch: "+str(count))
print("Ok: "+str(inverse_count))
print(preprocessed_df.shape)
#print(preprocessed_df.head())


Mismatch: 423
Ok: 25028
(25028, 4)


In [6]:
count = inverse_count = 0
positive_df = negative_df = None
for i in range(preprocessed_df.shape[0]): 
  if preprocessed_df['label'][i] == 1:
    count+=1
    if positive_df is None:
      positive_df = pd.DataFrame(preprocessed_df.iloc[i, :]).transpose()
    else:
      positive_df = pd.concat([positive_df, pd.DataFrame(preprocessed_df.iloc[i, :]).transpose()], ignore_index=True)
  else:
    inverse_count+=1
    # Append the row to preprocessed_df
    if negative_df is None:
      negative_df = pd.DataFrame(preprocessed_df.iloc[i, :]).transpose()
    else:
      negative_df = pd.concat([negative_df, pd.DataFrame(preprocessed_df.iloc[i, :]).transpose()], ignore_index=True)

positive_df = positive_df.drop_duplicates(subset=["Issue_id", "candidate_string"], keep='first')
negative_df = negative_df.drop_duplicates(subset=["Issue_id", "candidate_string"], keep='first')
print(positive_df.shape)
print(negative_df.shape)


(417, 4)
(24067, 4)


In [7]:

positive_chunks = np.array_split(positive_df, 2)
negative_chunks = np.array_split(negative_df, 2)

c:\Users\ahmed\miniconda3\envs\tf\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [8]:
Rater2_positive = positive_chunks[0].sample(n=100, replace=False)
Rater1_positive = positive_chunks[1].sample(n=100, replace=False)
Rater2_negative = negative_chunks[0].sample(n=100, replace=False)
Rater1_negative = negative_chunks[1].sample(n=100, replace=False)

In [9]:
Rater2_df = pd.concat([Rater2_positive, Rater2_negative], ignore_index=True)
Rater1_df = pd.concat([Rater1_positive, Rater1_negative], ignore_index=True)
Rater2_df = Rater2_df.sample(frac=1)
Rater1_df = Rater1_df.sample(frac=1)
print(Rater2_df.shape)
print(Rater1_df.shape)
Rater2_df.to_csv("../Kappa/2_df.csv")
Rater1_df.to_csv("../Kappa/1_df.csv")

(200, 4)
(200, 4)
